# Fake News Predication Using Random Forest Algorithm.

Fake news detection is a hot topic in the field of natural language processing. We consume news through several mediums throughout the day in our daily routine, but sometimes it becomes difficult to decide which one is fake and which one is authentic. Our job is to create a model which predicts whether a given news is real or fake.

# Project Flow:
    1. Problem Statement
    2. Data Gathering
    3. Data Preprocessing : Here we perform some operation on data
        A. Tokenization
        B. Lower Case
        C. Stopwords 
        D. Lemmatization / Stemming
    4. Vectorization (Convert Text data into the Vector):
        A. Bag Of Words (CountVectorizer)
        B. TF-IDF
    5. Model Building :
        A. Model Object Initialization
        B. Train and Test Model
    6. Model Evaluation :
        A. Accuracy Score
        B. Confusition Matrix
        C. Classification Report
    7. Model Deployment
    8. Prediction on Client Data        

# Required Libraries

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1.Data Gathering 

In [2]:
df=pd.read_csv("News_dataset.csv")

In [3]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


# Data Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [6]:
df.shape

(20800, 5)

In [7]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df=df.dropna() #Handling the missing values by droping those rows

In [9]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
df.shape

(18285, 5)

In [11]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [12]:
df.reset_index(inplace=True)
df.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [13]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [14]:
df['title'][0:10]

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
5    Jackie Mason: Hollywood Would Love Trump if He...
6    Benoît Hamon Wins French Socialist Party’s Pre...
7    A Back-Channel Plan for Ukraine and Russia, Co...
8    Obama’s Organizing for Action Partners with So...
9    BBC Comedy Sketch "Real Housewives of ISIS" Ca...
Name: title, dtype: object

In [15]:
df['author'][0:10]

0                   Darrell Lucus
1                 Daniel J. Flynn
2              Consortiumnews.com
3                 Jessica Purkiss
4                  Howard Portnoy
5                 Daniel Nussbaum
6                 Alissa J. Rubin
7    Megan Twohey and Scott Shane
8                     Aaron Klein
9                 Chris Tomlinson
Name: author, dtype: object

In [16]:
df=df.drop(['id','text','author'],axis=1) #droping the id,text,author

In [17]:
df.head()

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


# Data Preprocessing

## 1. Tokenization 
the process of spliting the string

In [18]:
sample_data = 'Alcarazs world No.1 title at risk after shock Canadian Open loss as Djokovic threat looms large in Cincinnati Masters'
sample_data=sample_data.split()
sample_data

['Alcarazs',
 'world',
 'No.1',
 'title',
 'at',
 'risk',
 'after',
 'shock',
 'Canadian',
 'Open',
 'loss',
 'as',
 'Djokovic',
 'threat',
 'looms',
 'large',
 'in',
 'Cincinnati',
 'Masters']

## 2.Make Lowercase
To re

In [19]:
sample_data = [data.lower() for data in sample_data ]
sample_data
len(sample_data)

19

## 3.Remove Stopwords

In [20]:
#To remove the Vocabulary of noise 

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [22]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [23]:
stopwords = stopwords.words('english')
print(stopwords[0:10])
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


In [24]:
sample_data=[data for data in sample_data if data not in stopwords]
print(sample_data)
len(sample_data)

['alcarazs', 'world', 'no.1', 'title', 'risk', 'shock', 'canadian', 'open', 'loss', 'djokovic', 'threat', 'looms', 'large', 'cincinnati', 'masters']


15

# 4. Stemming 

 Stemming is the process of reducing words to their base or root form

In [25]:
ps=PorterStemmer()
sample_data_stemming = [ps.stem(data)for data in sample_data]
print(sample_data_stemming)

['alcaraz', 'world', 'no.1', 'titl', 'risk', 'shock', 'canadian', 'open', 'loss', 'djokov', 'threat', 'loom', 'larg', 'cincinnati', 'master']


# 5.Lemmatization

In [26]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omkar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
 import nltk
from nltk.stem import WordNetLemmatizer

In [32]:
 import nltk
nltk.download('omw-1.4')

[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [28]:
lm = WordNetLemmatizer()
sample_data_lemma = [lm.lemmatize(data) for data in sample_data]
print(sample_data_lemma)

['alcarazs', 'world', 'no.1', 'title', 'risk', 'shock', 'canadian', 'open', 'loss', 'djokovic', 'threat', 'loom', 'large', 'cincinnati', 'master']


In [29]:
lm=WordNetLemmatizer()
corpus=[]
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9','',df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [30]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [31]:
len(corpus)

18285

In [32]:
corpus[0]

'house dem aide: didn’t even see comey’s letter jason chaffetz tweeted'

# 4.Vectorization (convert text data to the vector)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
tf = TfidfVectorizer()
x = tf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

# Data Spliting into the train and Test Case

In [36]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

In [37]:
len(x_train),len(y_train)

(12799, 12799)

In [38]:
len(x_test),len(y_test)

(5486, 5486)

# 5.Model Building

# Random Forest Classifier

In [39]:
rf = RandomForestClassifier()
# Create an instance of the RandomForestClassifier
rf.fit(x_train, y_train)
# Train the Random Forest classifier

RandomForestClassifier()

# Decision Tree classifier

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
# Create an instance of the DecisionTreeClassifier
dt = DecisionTreeClassifier()
# Train the Decision Tree classifier
dt.fit(x_train, y_train)

DecisionTreeClassifier()

# 6.Model Evalution

In [42]:
y_pred = rf.predict(x_test)
accuracy_score_ = accuracy_score(y_test,y_pred) 
accuracy_score_

0.9378417790740066

In [43]:
y_pred_dt= dt.predict(x_test)
accuracy_score_Dt= accuracy_score(y_test,y_pred) 
accuracy_score_Dt

0.9378417790740066

In [44]:
class Evaluation:
    def __init__(self,model,x_train,x_test,y_train,y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        
    def train_evaluation(self):
        y_pred_train = self.model.predict(self.x_train)
        
        acc_scr_train = accuracy_score(self.y_train,y_pred_train)
        print("Accuracy Score on Training Data Set:",acc_scr_train)
        print()
        
        con_mar_train = confusion_matrix(self.y_train,y_pred_train)
        print("Confusion Matrix on Training Data Set:\n",con_mar_train)
        print()
    
        
        class_rep_train = classification_report(self.y_train,y_pred_train)
        print("Classification Report on Training Data Set:\n",class_rep_train)
        print()
        
    def test_evaluation(self):
        y_pred_test = self.model.predict(self.x_test)
        
        acc_scr_test = accuracy_score(self.y_test,y_pred_test)
        print("Accuracy Score on Testing Data Set:",acc_scr_test)
        print()
        
        con_mar_test = confusion_matrix(self.y_test,y_pred_test)
        print("Confusion Matrix on Testing Data Set:\n",con_mar_test)
        print()
    
        
        class_rep_test = classification_report(self.y_test,y_pred_test)
        print("Classification Report on Testing Data Set:\n",class_rep_test)
        print()

In [45]:
#checking the accucary  on the training  dataset:
Evaluation(rf,x_train,x_test,y_train,y_test).train_evaluation()

Accuracy Score on Training Data Set: 1.0

Confusion Matrix on Training Data Set:
 [[7252    0]
 [   0 5547]]

Classification Report on Training Data Set:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7252
           1       1.00      1.00      1.00      5547

    accuracy                           1.00     12799
   macro avg       1.00      1.00      1.00     12799
weighted avg       1.00      1.00      1.00     12799




In [46]:
Evaluation(dt,x_train,x_test,y_train,y_test).train_evaluation()

Accuracy Score on Training Data Set: 1.0

Confusion Matrix on Training Data Set:
 [[7252    0]
 [   0 5547]]

Classification Report on Training Data Set:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7252
           1       1.00      1.00      1.00      5547

    accuracy                           1.00     12799
   macro avg       1.00      1.00      1.00     12799
weighted avg       1.00      1.00      1.00     12799




In [47]:
Evaluation(rf,x_train,x_test,y_train,y_test).test_evaluation()

Accuracy Score on Testing Data Set: 0.9378417790740066

Confusion Matrix on Testing Data Set:
 [[2827  282]
 [  59 2318]]

Classification Report on Testing Data Set:
               precision    recall  f1-score   support

           0       0.98      0.91      0.94      3109
           1       0.89      0.98      0.93      2377

    accuracy                           0.94      5486
   macro avg       0.94      0.94      0.94      5486
weighted avg       0.94      0.94      0.94      5486




In [48]:
Evaluation(rf,x_train,x_test,y_train,y_test).test_evaluation()

Accuracy Score on Testing Data Set: 0.9378417790740066

Confusion Matrix on Testing Data Set:
 [[2827  282]
 [  59 2318]]

Classification Report on Testing Data Set:
               precision    recall  f1-score   support

           0       0.98      0.91      0.94      3109
           1       0.89      0.98      0.93      2377

    accuracy                           0.94      5486
   macro avg       0.94      0.94      0.94      5486
weighted avg       0.94      0.94      0.94      5486




# predication Pipeline

In [49]:
class Preprocessing:
    def __init__(self,data):
        self.data=data
        
        
    def text_preprocessing_user(self):
        lm=WordNetLemmatizer()
        pred_data=[self.data]
        preprocess_data=[]
        for i in pred_data:
            review = re.sub('^a-zA-Z0-9',' ',data)
            review=review.lower()
            review=review.split()
            review = [lm.lemmatize(x) for x in review if x not in stopwords]
            review = " ".join(review)
            preprocess_data.append(review)
        return preprocess_data

In [50]:
df['title'][10]

'Russian Researchers Discover Secret Nazi Military Base ‘Treasure Hunter’ in the Arctic [Photos]'

In [51]:
data='Russian Researchers Discover Secret Nazi Military Base ‘Treasure Hunter’ in the Arctic [Photos]'
Preprocessing(data).text_preprocessing_user()

['russian researcher discover secret nazi military base ‘treasure hunter’ arctic [photos]']

In [52]:
class Prediction:
    
    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model
        
    def prediction_model(self):
        preprocess_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocess_data)
        prediction = self.model.predict(data)
        
        if prediction [0] == 0 :
            return "The News Is Fake"
        
        else:
            return "The News Is Real"    

In [59]:
data= 'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - BreitbartA Back-Channel Plan for Ukraine and Russia, Courtesy of Trump Associates - The New York Times'
Prediction(data,rf).prediction_model()

'The News Is Fake'

In [61]:
data= 'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
Prediction(data,rf).prediction_model()

'The News Is Real'

In [60]:
df['title'][0]


'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [63]:
user_data='India s Chandrayaan-3 lunar mission: Fake news takes flight alongside spacecraft'
Prediction(user_data,rf).prediction_model()

'The News Is Real'

In [ ]:
user_data='Chief Minister’s Claim On No Riots In Uttar Pradesh Since 2017'
Prediction(user_data,rf).prediction_model()